In [ ]:
!pip install langchain langchain-google-genai python-dotenv langchain-groq

### Load API Key

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

gemini_api_key  

### Use Models

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    max_retries=1,
    temperature=1
)

In [ ]:
response = model.invoke("Hello, What is use of GPU in Deep learing, explain in 3 sentences")

In [ ]:
print(response.content)

In [ ]:
response.usage_metadata

### Context aware generation

In [ ]:
system_prompt = """
You are a helpful and polite support assistant for **Lumbini ICT Campus**.

Your role is to assist students by answering questions **only related to Lumbini ICT Campus**.
You may respond to simple greetings.

If a user asks any question that is **not related to the campus**, clearly reply:
"I don't know."

If a user asks for information about the campus that is **not included below**, politely ask them to contact the college directly at:
+977 9812345678

---

### Campus Information

**Location**
- Kaligandaki Chowk, Gaindakot-5, Nawalpur

**Available Courses**
- BSc. CSIT
- BCA
- BIM
- BHM

**Course Details**
- **BSc. CSIT**: 4 years, only students with a science background in +2
- **BCA**: 4 years, students from any background
- **BIM**: 3 years, only students with a management background
- **BHM**: 4 years, students from any background

**Campus Advantages**
- Regular extra classes
- Workshops
- Sports events
- Hackathons
"""

In [ ]:
from langchain.messages import HumanMessage, SystemMessage


query = input("Ask anything about campus")

response = model.invoke(
    [
        SystemMessage(content=system_prompt),
        HumanMessage(query)
    ]
)

In [ ]:
response.content

### Tool calling

In [ ]:
from langchain.tools import tool
import json

@tool
def search_database():
    """
    Search the products database to see records of `products`
    """
    records = [
        {
            "id": 1,
            "name": "Phone Case",
            "price": 800,
            "available": True
        },
        {
            "id": 2,
            "name": "Charger",
            "price": 3200,
            "available": True
        },
        {
            "id": 3,
            "name": "Power bank",
            "price": 6000,
            "available": False,
            "next_available_date": "2026-01-01"
        }
    ]
    return json.dumps(records)

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[search_database]
)

In [ ]:
from langchain.messages import HumanMessage


messages = [HumanMessage("hi")]

response = agent.invoke({"messages": messages})

In [ ]:
response["messages"][-1]

In [ ]:
response = agent.invoke({"messages": HumanMessage("What is price of charger")})

In [ ]:
response

In [ ]:
response["messages"][-1].content

In [ ]:
agent.invoke({"messages": "Who is current PM of USA"})